### Importing libraries

In [75]:
import pandas as pd
import altair as alt
import numpy as np
from scipy.integrate import quad
from scipy.stats import norm
from scipy.optimize import minimize

In [76]:
alt.themes.enable('dark')
alt.data_transformers.disable_max_rows()
alt.warnings.simplefilter(action='ignore')

### Read CSV (ISC)

In [92]:
EQ_data = pd.read_csv("ISC_catalog_1950_1999.csv", usecols=[0,1,2,3,4,5,6,7,8,9,10,11])
EQ_data = EQ_data.applymap(lambda x: x.strip() if isinstance(x, str) else x)
EQ_data['MAG'] = EQ_data['MAG'].astype('float32')


### Highlight nuclear explosions

In [93]:
# mask = ((EQ_data['TYPE'] == 'kn') & (EQ_data['MAG'].isna() != True))
# EQ_data = EQ_data[mask]
EQ_data = EQ_data.dropna(subset=['MAG'])
# EQ_data['MAGTYPE'].value_counts()

### Specify mag type

In [94]:
EQ_data['MAGTYPE'].value_counts()
EQ_data = EQ_data[EQ_data['MAGTYPE'].isin(EQ_data['MAGTYPE'].value_counts().index[0:17])]

In [95]:
prev_EQ = EQ_data

In [96]:
def getYear(time):
    return int(str.split(time, '-')[0])

EQ_data['YEAR'] = EQ_data['DATE'].apply(getYear)

In [97]:
EQ_data.groupby('YEAR')['MAGTYPE'].value_counts()

YEAR  MAGTYPE
1950  M          208
      MS         188
1951  MS         211
      M           75
1952  MS         270
                ... 
1999  Md         307
      MN         259
      mbLg        90
      Mb          45
      MLH         12
Name: count, Length: 359, dtype: int64

In [101]:
mag_year_data = EQ_data.groupby('YEAR')['MAGTYPE'].value_counts()
selection = alt.selection_multi(fields=['MAGTYPE'], bind='legend')
hist_MY = alt.Chart(mag_year_data.reset_index()).mark_bar().add_selection(selection).encode(
    alt.X('YEAR'),
    alt.Y('count'),
    color='MAGTYPE',
    opacity=alt.condition(selection, alt.value(1), alt.value(0))
).interactive()
hist_MY

alt.Chart(...)

In [175]:
EQ_data = prev_EQ
# EQ_data = EQ_data[EQ_data['MAGTYPE'].isin(['mb', 'ms', 'mwc', 'mwb', 'uk'])]
# EQ_data = EQ_data[EQ_data['MAGTYPE'].isin(['MB', 'Mb', 'mL', 'M', 'ML', 'MD', 'ml' ])]
EQ_data = EQ_data[EQ_data['MAGTYPE'].isin(['M', 'ML', ])]

### Plot GR graph

In [176]:
Y_cumulative = []
GR_source = EQ_data['MAG'].value_counts().reset_index().sort_values(by='MAG')

for m in GR_source['MAG']:
    Y_cumulative.append(np.sum(GR_source[GR_source['MAG'] >= m]['count']))

source = pd.DataFrame({
  'M': GR_source['MAG'],
  'logN': np.log10(GR_source['count']),
  'logN (cumulative)': np.log10(Y_cumulative) 
})
GR_chart = alt.Chart(source.melt('M')).mark_point(filled=True).encode(
    x='M',
    y='value',
    color='variable',
).interactive()

In [177]:
hist_MN = alt.Chart(EQ_data['MAG'].value_counts().reset_index()).mark_bar().encode(
    alt.X('MAG'),
    alt.Y('count')
).interactive()
hist_NY = alt.Chart(EQ_data['YEAR'].value_counts().reset_index()).mark_bar().encode(
    alt.X('YEAR'),
    alt.Y('count')
)


hist_MN | GR_chart | hist_NY

alt.HConcatChart(...)

## Estimate magnitude of completeness

### OK1993 method

In [178]:
def q(m, mu, sig):
    return quad(norm.pdf, -np.inf, m, args=(mu, sig))[0]

def f(m, b, mu, sig):
    return b*np.exp((-b)*(m-mu) - (b*sig)**2/2) * q(m, mu, sig)

def neg_logL(parameters):
    b, mu, sig = parameters
    res = np.sum([np.log(f(m, b, mu, sig))*count for m, count in EQ_data['MAG'].value_counts().items()])
    return -res

In [179]:
# neg_logL([1,5,1])

In [180]:
# mle_model = minimize(neg_logL, np.array([1,0,1]), method='')
# mle_model

### b−value stability method (MBS)

In [181]:
delta_m = 0.1   # bin size
dM = 0.5        # in b_ave

In [182]:
def frange(x, y, jump):
    while x < y:
        yield x
        x += jump

def preCalc(Data):

    b_list = {}
    db_list = {}

    for M_c in frange(0.0, 10.0, delta_m):
        M_c = round(M_c, 2)
        Sample = Data[Data['MAG'] >= M_c]['MAG']
        M_mean = Sample.mean()
        N = Sample.size

        b = np.log10(np.e)/(M_mean - M_c + delta_m/2)
        db = 2.3*b**2*np.sqrt(np.sum((Sample - M_mean)**2)/N/(N-1))

        b_list[M_c] = b
        db_list[M_c] = db
    
    return b_list, db_list

In [183]:
def calcMc(b_list, db_list):

    for M_c in frange(0.0, 9.0, delta_m):
        M_c = round(M_c, 2)
        b = b_list[M_c]
        db = db_list[M_c]

        num = 0
        b_ave = 0
        for M_co in frange(M_c, M_c+dM, delta_m):
            M_co = round(M_co, 2)
            b_ave += b_list[M_co]
            num += 1
        b_ave /= num    

        if abs(b_ave - b) <= db:
            break
    
    return M_c, b

In [184]:
b_list, db_list = preCalc(EQ_data)
M_c, b = calcMc(b_list, db_list)

M_c, b

(3.8, np.float64(0.9615249382108927))

In [185]:
a = np.log10(EQ_data[abs(EQ_data['MAG'] - M_c) < delta_m/2].shape[0])

In [186]:
x = np.linspace(M_c-1, 9, 100)
source = pd.DataFrame({'x': x, 'f(x)': a - b*(x-M_c)})

GR_line = alt.Chart(source).mark_line(
    color='cornsilk',
    strokeWidth=1.5
).encode(
    x='x',
    y='f(x)'
)

rule = alt.Chart(pd.DataFrame({
  'M': [M_c],
})).mark_rule(
    color="darksalmon",
    strokeWidth=1.5,
    strokeCap='round',
    strokeDash=[8,8],
).encode(x='M')

In [187]:
hist_MN | GR_line + (GR_chart+rule) | hist_NY

alt.HConcatChart(...)

### Cutoff magnitude evolution over time

In [188]:
def getMc(year):
    b_list, db_list = preCalc(EQ_data[EQ_data['YEAR'] <= year])
    M_c, b = calcMc(b_list, db_list)
    return M_c, b

In [189]:
yMc_data = pd.DataFrame({'year': EQ_data['YEAR'].unique()})

bMc_data = []
for y in yMc_data['year']:
    bMc_data.append(getMc(y))


In [190]:
yMc_data['M_c'] = np.array(bMc_data, ndmin=2)[:,0]
yMc_data['b'] = np.array(bMc_data, ndmin=2)[:,1]

In [191]:
Mc_chart = alt.Chart(yMc_data).mark_line(
    color='cornsilk',
    strokeWidth=1.5
).encode(
    x='year', 
    y=alt.Y('M_c', )#scale=alt.Scale(domain=(4,7)))
)

b_chart = alt.Chart(yMc_data).mark_line(
    color='lightblue',
    strokeWidth=1.5
).encode(
    x='year', 
    y=alt.Y('b',)# scale=alt.Scale(domain=(0.7,1.5)))
).interactive()

Mc_chart | b_chart

alt.HConcatChart(...)